<a href="https://colab.research.google.com/github/ragnor-devop/Creditos_v07/blob/main/CuadernoProduccion_v067.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Learning para la aprobacion de creditos en entidades financieras**





###Importacion librerias

In [ ]:
#Librerias para manipulacion de datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# plt.style.use("dark_background")
#Librerias para metricas
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score
import time
#Librerras para Preprocesado y holdout
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
#Librerias de los modelos a utilizar
from sklearn.tree import export_graphviz,DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import cohen_kappa_score
#Para la implementacion
import warnings 
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display, clear_output
import time
from datetime import datetime
import codecs
import io
import sys
import base64


###Importacion el Data Set

In [ ]:
#Se obtiene el Dataset en un pandas.DataFrame almacenado en la variable df.
# df=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/demo_day_data_sets/main/01%20Data_Set_Banco/01%20credit_train_v03.csv")
# df=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/credit_train_05.csv")
df=pd.read_csv("https://raw.githubusercontent.com/ragnor-devop/CreditosDataset01/main/DataSets/credit_train_v06.csv")
df=df.drop(columns=["Unnamed: 0"])

##Preparacion del Modelo


In [ ]:
  #Eliminar Columnas Prestamo ID Cliente ID
  df=df.drop(columns=["Prestamo ID","Cliente ID"])
  #Eliminacion los registros duplicados 
  df=df.drop_duplicates() 
  #Se rellena los vacios con la media
  df=df.fillna(df.mean())
  #Transformar la columna "Years in current job"de object en float al extraer los caracteres no numericos
  df['Años en trabajo actual']=df['Años en trabajo actual'].str.extract(r"(\d+)")
  df['Años en trabajo actual'] =df['Años en trabajo actual'].astype(float)
  #Eliminar los registros nulos 
  df=df.dropna()
  #Transformar valores categoricos a valores numericos
  columnas=df.columns
  y=pd.DataFrame(data=df["Estado de prestamo"],columns=["Estado de prestamo"])
  df=df.drop(columns=["Estado de prestamo"])
  categoricos = ['Termino','Propiedad de vivienda','Proposito']
  ycategorico=['Estado de prestamo']
  ctx = ColumnTransformer([('transform', OneHotEncoder(), categoricos )],  remainder = 'passthrough')
  cty = ColumnTransformer([('transform', OneHotEncoder(), ycategorico )],  remainder = 'passthrough')
  ty=  np.array(cty.fit_transform(y), dtype = np.float)
  t=  np.array(ctx.fit_transform(df), dtype = np.float)
  # Nueva forma del dataset
  y=pd.DataFrame(ty)
  df=pd.DataFrame(t)
  # Renombrar los nombres de las columnas
  rename={0:'Largo plazo',1:'Corto Plazo',2:'Tiene hipoteca',3:'Casa con Hipoteca',4:'Casa Propia', 5:'Alquiler'
          ,6:'Compra vehiculo', 7:'Construccion', 8:'Consumo', 9:'Educacion',10:'Microcredito'
          ,11:'Otros', 12:'Pagar deuda'
          ,13:'Pyme', 14:'Vivienda'
          ,15:"Monto de prestamo", 16:'Valor de credito',17:'Ingreso Anual',18:'Años en trabajo actual',19:'Deuda Mensual'
          ,20:'Historial Crediticio',21:'Meses desde la ultima mora',22:'Numero de cuentas abiertas',23:'Nro de problemas crediticios'
          ,24:'Balance de credito actual',25:'Maximo monto de credito',26:'Quiebras',27:'Gravamenes fiscales', }
  df.rename(columns=rename,inplace=True)

  yrename={0:'Descargado',1:'Pagado completamente'}
  y.rename(columns=yrename,inplace=True)

  #Eliminacion de las variables ficticias
  df.drop(columns=['Largo plazo', 'Alquiler',"Pyme"], axis=1, inplace=True)
  y.drop(columns=['Descargado'],axis=1,inplace=True)
  X=df.copy()
  #Division del conjunto de datos
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=33)

  # Se recupera en Xcolumns los nombres de las columnas
  Xcolumnas=X_train.columns
  # Normalizacion de los datos
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  #Lo mismo para los datos de testeo
  X_test = scaler.transform(X_test)

  # Transformar a un pandas.DataFrame los datos de entrenamiento
  X_train=pd.DataFrame(data=X_train,columns=Xcolumnas)
  X_test=pd.DataFrame(data=X_test,columns=Xcolumnas)


In [ ]:
#Entrenamiento del modelo Logistic Regression
logistic_regresion = LogisticRegression(max_iter=20000,class_weight="dict",solver="lbfgs")
lr_entrenamiento_tiempo_inicio= time.time()
logistic_regresion.fit(X_train, y_train)
lr_entrenamiento_tiempo_fin = time.time()
logistic_regression_tiempo_entrenamiento = lr_entrenamiento_tiempo_fin - lr_entrenamiento_tiempo_inicio # in seconds
# print(f"Tiempo de ejecucion para Entrenamiento Logistic Regression: {logistic_regression_tiempo_entrenamiento:,.3f} ")

# prediccion Logistic Regression
lr_predict_tiempo_inicio= time.time()
y_pred_regression = logistic_regresion.predict(X_test)
lr_predict_tiempo_fin = time.time()
logistic_regression_tiempo_prediccion = lr_predict_tiempo_fin - lr_predict_tiempo_inicio # in seconds
# print(f"Tiempo de ejecucion para Prediccion Logistic Regression: {logistic_regression_tiempo_prediccion:,.3f} ")

## Metodos Para Procesado del Conjunto de Datos

In [ ]:
def displaydf(dataframe):
    display(dataframe)
    return True

In [ ]:
def mostrarGrafica(dataframe):
  
  fig = plt.figure(figsize=(15 ,15))
  fig.canvas.capture_scroll = True
  fig.canvas.toolbar_visible = True
  plt.plot(np.sin(np.linspace(0, 20, 100)));
  # dataframe['Resultado'].hist()
  # dataframe.groupby('Resultado')['Termino'].nunique().plot(kind='bar')
  display(fig.canvas)
  # categoria = ['Estado de prestamo', 'Termino', 'Años en trabajo actual', 'Propiedad de vivienda']
  # plt.figure(figsize=(15 ,15))
  # for i in range(1,4):
  #   plt.subplot(2,2,i)
  #   sns.countplot(x=categoria[i], data =df, hue='Estado de prestamo')
  # plt.show()

In [ ]:
def limpiezaDataSet(ldf,recuperarN):
  #Eliminacion los registros duplicados 
  recuperarN = ldf.drop(columns=["Cliente ID","Nombre"])
  ldf=ldf.drop(columns=["Cliente ID","Nombre"])
  # ldf=ldf.drop(columns=["Unnamed: 0"])
  return ldf,recuperarN




In [ ]:
def preprocesadoDataSet(predf,prect,prescaler):
  #Transformar valores categoricos a valores numericos
  
  pret=  np.array(prect.transform(predf), dtype = np.float)
  # Nueva forma del dataset
  predf=pd.DataFrame(pret)
  # Renombrar los nombres de las columnas con proposito
  prerename={0:'Largo plazo',1:'Corto Plazo',2:'Tiene hipoteca',3:'Casa con Hipoteca',4:'Casa Propia', 5:'Alquiler'
        ,6:'Compra vehiculo', 7:'Construccion', 8:'Consumo', 9:'Educacion',10:'Microcredito'
        ,11:'Otros', 12:'Pagar deuda'
        ,13:'Pyme', 14:'Vivienda'
        ,15:"Monto de prestamo", 16:'Valor de credito',17:'Ingreso Anual',18:'Años en trabajo actual',19:'Deuda Mensual'
        ,20:'Historial Crediticio',21:'Meses desde la ultima mora',22:'Numero de cuentas abiertas',23:'Nro de problemas crediticios'
        ,24:'Balance de credito actual',25:'Maximo monto de credito',26:'Quiebras',27:'Gravamenes fiscales' }
  predf.rename(columns=prerename,inplace=True)
  
  #Eliminacion de las variables ficticias CON PROPOSITO
  predf.drop(columns=['Largo plazo', 'Alquiler',"Pyme"], axis=1, inplace=True)
  
  # Se recupera en Xcolumns los nombres de las columnas
  preXcolumnas=predf.columns

  #Lo mismo para los datos de testeo
  predf = prescaler.transform(predf)

  # Transformar a un pandas.DataFrame los datos de entrenamiento
  predf=pd.DataFrame(data=predf,columns=preXcolumnas)
  
  
  return predf


In [ ]:
def procesado(archivo_csv, estado):
  """
  "Realiza la preparacion de los datos"

  archivo_csv   = archivo en formato csv
  estado        = una cadena donde se registra el estado
  ## Retorno ##
  df      pandas.dataframe    conjunto de datos preprocesado
  estado  str                 
  
  """
  dfprepro=dfs=pd.DataFrame(data=[""],columns=[""])
  estadof=""
  try:
    
    uploaded_file=archivo_csv.data[0]
    dfs=pd.read_csv(io.BytesIO(uploaded_file))
    dfs2=dfs.copy()
    
     #CODIGO PARA PREPROCESAR EL DATASET almacenado en df
    dfr,recN=limpiezaDataSet(dfs,[0,1])
    dfprepro=preprocesadoDataSet(dfr,ctx,scaler)
    dfp,estadof = analisisDatos(dfs2,dfprepro,estado)
  except Exception as ex:
    estadof+="Archivo con formato incorrecto"
  
  # dfp,estadof = analisisDatos(dfs2,dfprepro,estado)
  
  return dfp,estadof


In [ ]:
def analisisDatos(dffinal,dfa, resultado):
  """
  El modelo analiza el conjunto de datos

  Parametros 
  df =        pandas.dataframe    El conjunto de datos dene estar procesado
  resultado = str                 Una cadena vacia

  Retorno

  df =        pandas.datafrana    El conjunto de datos con una columna adicional de valores predichos
  resultado   str                 Una cadena con el estado del proceso

  Errores
  SI el conjunto de datos no esta preprocesado
  """
  tiempo_final=0
  try:
    tiempo_inicio= time.time()
    prediccion=logistic_regresion.predict(dfa)
    tiempo_fin = time.time()
    tiempo_final= tiempo_fin - tiempo_inicio # in seconds

    dffinal["Resultado"]=prediccion
    # mostrarGrafica(dffinal)
    dffinal.loc[dffinal["Resultado"]==1,"Aprobar"]="Aprobar"
    dffinal.loc[dffinal["Resultado"]==0,"Aprobar"]="No Aprobar"
    
    estado="Procesado Exitoso"
  except Exception as ex:
    resultado = resultado + "Error al analizar Conjunto" +"\n "

  return dffinal,resultado
  

##**Implementacion**


In [ ]:
def generarBoton(res,estado,fechainf):
  #FILE
  res=res.to_csv()
  fechastr=f"{fechainf.year}-{fechainf.month}-{fechainf.day}"
  filename = f'informe-{fechastr}.csv'
  b64 = base64.b64encode(res.encode())
  payload = b64.decode()

  #BUTTONS
  html_buttons = '''<html>
  <head>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  </head>
  <body>
  
  <a download="{filename}" href="data:text/csv;base64,{payload}" download>
  <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Descargar Informe</button>
  </a>
  
  </body>
  </html>
  '''

  html_button = html_buttons.format(payload=payload,filename=filename)
  display(HTML(html_button))

In [ ]:
fecha = widgets.DatePicker(value=datetime.now())
archivo=widgets.FileUpload(
    accept='.csv',
    multiple=False)  # True to accept multiple files upload else False
# button send
button_send = widgets.Button(
                description='Generar Informe',
                button_style='success',
                tooltip='Send',
                style={'description_width': 'initial'},
                icon='angle-down'
                
            )
# button_send.style.button_color='success'

output = widgets.Output()

def on_button_clicked(event):
    with output:

      try:
        clear_output()
        res,estado = procesado(archivo,"")
        print(estado)
        generarBoton(res,estado,fecha.value)
        display(res.head(10))
      except Exception as ex:
        display("Error en cargar archivo intentar nuevamente")

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])


In [ ]:
# stacked right hand side

text_0 = widgets.HTML(value='<h1 style="color:teal;">Modelo de Machine Learning</h1>')
text_1 = widgets.HTML(value='<h2 style="font-style: italic;color:#003399;">Regresion Logistica</h2>')
text_2= widgets.HTML(value='<h3 style="color:teal;">Fecha informe</h3>')
text_3= widgets.HTML(value='<h3 style="color:teal;">Subir Archivo:</h3>')

vbox_fecha=widgets.VBox([text_2,fecha])
vbox_text = widgets.VBox([text_0, text_1,vbox_fecha,text_3, archivo, vbox_result])
page = widgets.VBox([vbox_text])
display(page)